# Using UNET for data segmentation

UNET es una red neuronal profunda utilizada para la segmentacion de imagenes, nacida para propositos biomedicos, la imagen de entrada es procesada mediante un codificador convolucional, hasta saturarla en las capas intermedias. Acto seguido, un decodificador convolucional convierte la imagen en una mascara mediante discriminacion cada una de las clases.

Aqui podemos ver la arquitectura de la red:
![U-Net architechture](images/u-net-architecture.png)

En esta implementacion, se realiza una adaptacion de la red para un dataset de imagenes de carreteras extraida de las camaras de varios coches autonomos.

Dicho dataset es accesible y descargable desde http://www.tromai.icoc.me/ y cargado y preprocesado mediante el metodo load_data del archivo loader.py

Este dataset contiene 20 clases etiquetadas cuyo groundtruth contiene las mascaras de las mismas. Las 20 clases son:
![Dataset Classes](images/dataset_classes.jpg)

In [1]:
from loader import load_data
print(load_data.__doc__)
(X_train, y_train, X_test, y_test) = load_data()


    Loads data into numpy arrays ready for Keras training / testing
    Returns: Tuple with train and test datasets
    


Una vez ha sido cargado el dataset, podemos ver las imagenes y las marcas mediante el uso de matplotlib, como podemos ver, la categorizacion final se realiza mediante el uso de la funcion argmax de la libreria numpy, que nos devuelve la clase dominante en cada pixel de la imagen.

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1,2,1)
plt.imshow(X_train[300])
plt.subplot(1,2,2)
plt.imshow(np.argmax(y_train[300], axis=2))

plt.show()
plt.clf()

plt.subplot(1,2,1);
plt.imshow(X_train[20]);
plt.subplot(1,2,2);
plt.imshow(np.argmax(y_train[20], axis=2))

plt.show()
plt.clf()

La red neuronal profunda utilizada ha sido extraida de https://github.com/petrosgk/Kaggle-Carvana-Image-Masking-Challenge, dicha red y los losses utilizados para el entrenamiento podemos encontrarla en los archivos recogidos en la carpeta model/, en el siguiente paso, realizamos la carga de la misma, concretamente de la red cuyo tamaño de entrada coincide con el de las imagenes del dataset utilizado, la topologia es volcada a un fichero JSON para el uso de la red en produccion.


In [ ]:
from model.u_net import get_unet_128 as unet

model = unet(num_classes=20)
print(model.summary())

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

Una vez tenemos los datos y la red preparados, podemos proceder al entrenamiento, la red sale del metodo ya compilada y lista para ser entrenada, por lo que unicamente tenemos que preparar los Callbacks de Keras utilizados para tunear el modelo y lanzar el entrenamiento. Una vez finalizado, guardamos los pesos.

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               min_delta=1e-4)]

entrenamiento = model.fit(X_train, y_train, batch_size=16, epochs=100, callbacks=callbacks, validation_data=(X_test, y_test))          

model.save_weights('model_weights.h5')

Una vez concluido el entrenamiento, lanzamos una grafica para comprobar como ha reducido la perdida de entrenamiento y validacion a lo largo de las iteraciones.

In [ ]:
ent_loss = entrenamiento.history['loss']
val_loss = entrenamiento.history['val_loss']

epochs = range(1, len(ent_loss) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, ent_loss, 'b', label='Entrenamiento')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'r', label='Validación')
plt.title('Pérdida en Entrenamiento y Validación')
plt.xlabel('Epochs')
plt.ylabel('Pérdida')
plt.legend()

plt.show()

Por ultimo, sacamos de una imagen, su groundtruth y la prediccion realizada de la misma.

In [ ]:
predicted = model.predict(X_test)

fig = plt.figure(figsize=(8,8))
fig.add_subplot(1, 3,1)
plt.imshow(X_test[0])
fig.add_subplot(1,3 ,2)
plt.imshow(np.argmax(y_test[0], axis=2))
fig.add_subplot(1,3,3)
plt.imshow(np.argmax(predicted[0], axis=2))

fig.show()